Code for the final project

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import librosa as lb
import matplotlib.pyplot as plt
import IPython.display as ipd
import scipy.io as sio
import warnings
from matplotlib.colors import LogNorm
from pydub import AudioSegment
from pydub.silence import split_on_silence 

In [3]:
warnings.filterwarnings(action='ignore') # to suppress warning messages

In [5]:
testing, sr = lb.core.load("audio/Test.mp3")

In [6]:
ipd.Audio(testing, rate = 22050)

In [42]:
Y

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [6]:
jimboi = AudioSegment.from_mp3("audio/Test.mp3")

In [7]:
audio_chunks = split_on_silence(jimboi, min_silence_len=500)

In [8]:
audio_chunks


[]

In [9]:
four_seconds = 4*1000;

In [10]:
first_4_seconds = jimboi[four_seconds:]

In [11]:
from pydub.playback import play


In [12]:
play(first_4_seconds)

In [13]:
first_4_seconds.dBFS

-34.81865938462336

In [14]:
audio_chunks = split_on_silence(jimboi, silence_thresh=-50)

In [15]:
len(audio_chunks)

4

In [16]:
play(audio_chunks[0])

In [17]:
play(audio_chunks[1])

In [18]:
play(audio_chunks[2])

In [54]:
play(audio_chunks[3])

In [26]:
samples = audio_chunks[3].get_array_of_samples()
samples = np.array(samples)

In [46]:
audio_chunks[3].channels

1

In [36]:
samples = samples.reshape(audio_chunks[3].channels, -1, order='F')

In [41]:
samples

array([[  -8,   17,    5, ..., -159,  -97,  -36]], dtype=int16)

In [37]:
samples.shape

(1, 15567)

In [35]:
ipd.Audio(samples, rate = 44100)

In [49]:
audio_chunks[3].export('export.mp3', format='mp3')

<_io.BufferedRandom name='export.mp3'>

In [50]:
Y, sr = lb.core.load("export.mp3")

In [51]:
Y

array([ -4.98869776e-05,  -1.89476723e-05,   5.68454698e-06, ...,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00], dtype=float32)

In [52]:
ipd.Audio(Y, rate = 22050)

In [57]:
def audiosegment_to_ndarray(audiosegment):
    samples = audiosegment.get_array_of_samples()
    samples_float = lb.util.buf_to_float(samples,n_bytes=2,
                                      dtype=np.float32)
    if audiosegment.channels==2:
        sample_left= np.copy(samples_float[::2])
        sample_right= np.copy(samples_float[1::2])
        sample_all = np.array([sample_left,sample_right])
    else:
        sample_all = samples_float
        
        
    return [sample_all,audiosegment.frame_rate]

In [60]:
test1 = audiosegment_to_ndarray(audio_chunks[3])

In [69]:
len(Y)
Y

array([ -4.98869776e-05,  -1.89476723e-05,   5.68454698e-06, ...,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00], dtype=float32)

In [71]:
test1[0]

array([-0.00024414,  0.0005188 ,  0.00015259, ..., -0.00485229,
       -0.00296021, -0.00109863], dtype=float32)

In [72]:
ipd.Audio(test1[0], rate = 22050)

In [76]:
samples = audio_chunks[3].get_array_of_samples()

In [78]:
samples_float = lb.util.buf_to_float(samples,n_bytes=2,dtype=np.float32)
samples_float

array([-0.00024414,  0.0005188 ,  0.00015259, ..., -0.00485229,
       -0.00296021, -0.00109863], dtype=float32)

In [81]:
import os
os.remove("export.mp3")

In [7]:
ree = lb.effects.split(testing, top_db=30)
ree[0]

array([11776, 26624])

In [8]:
testing.shape

(177408,)

In [9]:
ipd.Audio(testing[11776:26624], rate = 22050)

In [137]:
ipd.Audio(testing[26624:], rate = 22050)

In [11]:
S_full, phase = lb.magphase(lb.stft(testing))

In [13]:
S_filter = lb.decompose.nn_filter(S_full)

In [14]:
S_filter = np.minimum(S_full, S_filter)

In [18]:
margin_i, margin_v = 2, 10
power = 2

mask_i = lb.util.softmask(S_filter,
                               margin_i * (S_full - S_filter),
                               power=power)
mask_v = lb.util.softmask(S_full - S_filter,
                               margin_v * S_filter,
                               power=power)

In [30]:
oneseventhreeeight, sr = lb.core.load("1738.mp3")

In [31]:
ipd.Audio(oneseventhreeeight, rate = 22050)

In [62]:
split = lb.effects.split(oneseventhreeeight, top_db=30)
split

array([[ 18432,  27648],
       [ 68096,  78848],
       [115712, 123904],
       [154624, 160256]])

In [66]:
ipd.Audio(oneseventhreeeight[154624:160256], rate = 22050)

In [19]:
S_foreground = mask_v * S_full


In [45]:
foreground = S_foreground*np.exp(1j*phase)

In [51]:
foregroundNoise = lb.core.istft(foreground, hop_length = 128)
foregroundNoise

array([ -1.65098601e-08,   1.22732092e-06,   1.30998569e-06, ...,
         7.99408645e-06,   3.88781700e-06,   3.13920395e-06], dtype=float32)

In [58]:
ipd.Audio(foregroundNoise, rate = 22050)

In [ ]:
#samples = samples.reshape(song.channels, -1, order='F'); samples.shape # (<probably 2>, <len(song) in samples>)